<a href="https://colab.research.google.com/github/Muhaiminul8473/Bengali-automated-notes-generation-service/blob/main/Keras_model_for_bengalinoteGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
from tensorflow import keras


In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt


     |████████████████████████████████| 71kB 8.3MB/s 


In [ ]:
from sklearn.metrics import precision_score,recall_score,classification_report

In [ ]:
import random
import numpy as np

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import keras.utils as ku
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from keras.layers import Dropout
from keras import optimizers

from numpy.random import seed


layers = keras.layers
models = keras.models
from kerastuner.tuners import Hyperband

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
print("Processing the Intents.....")
with open('/content/gdrive/My Drive/Final defence/intents3.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [ ]:
print(words)

['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত', 'শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে', 'বিএনপি', 'অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে', 'সমাবেশ', 'করবে', 'বিএনপি', 'ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়', 'আগুন', 'প্লাস্টিক', 'কারখানায়', 'আগুন', 'ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়', 'ফকিরাপুলে', 'আবাসিক', 'হোটেলে', 'খুন', 'আবাসিক', 'হোটেলে', 'খুন', 'ফকিরাপুলে', 'আবাসিক', 'হোটেলে', 'শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ', 'অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ', 'শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড', 'নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', ':', 'নাসিম', 'নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', ':', 'নাসিম', 'জামালপুরে', 'বিদ্যুৎস্পৃষ্ট', 'হয়ে', 'কলেজছাত্রীর', 'মৃত্যু', 'কলেজছাত্রীর', 'মৃত্যু', 'জামালপুরে', 'বিদ্যুৎস্পৃষ্ট', 'হয়ে', 'ঢাকা-চট্টগ্রাম', 'ও', 'ঢাকা-ময়মনসিংহ', 'মহাসড়কের', '

In [ ]:
print(documents)

[(['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত'], 'নিহত'), (['চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত'], 'নিহত'), (['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়'], 'নিহত'), (['অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে', 'বিএনপি'], 'বিএনপি'), (['অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে'], 'বিএনপি'), (['সমাবেশ', 'করবে', 'বিএনপি'], 'বিএনপি'), (['ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়', 'আগুন'], 'আগুন'), (['প্লাস্টিক', 'কারখানায়', 'আগুন'], 'আগুন'), (['ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়'], 'আগুন'), (['ফকিরাপুলে', 'আবাসিক', 'হোটেলে', 'খুন'], 'খুন'), (['আবাসিক', 'হোটেলে', 'খুন'], 'খুন'), (['ফকিরাপুলে', 'আবাসিক', 'হোটেলে'], 'খুন'), (['শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ'], 'বিলবোর্ড'), (['অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ'], 'বিলবোর্ড'), (['শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড'], 'বিলবোর্ড'), (['নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', ':', 'নাসিম'], 'নির্বাচন'), (['নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়'], 'নির্বাচন'), (['

In [ ]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
1441 documents
383 classes ['', ' অঙ্ক', ' অপেক্ষা', ' উপহার', ' এক', ' গোল্ডকাপ', ' চাপা', ' চিরস্থায়ী', ' দৈত্য', ' ধ্বংস', ' ফেসবুক', ' ফ্লাইট', ' ব্যতিব্যস্ত', ' ভর্তি', ' লিনাক্সে', ' সেমিনার', ' স্বার্থ', ' হ্যাটট্রিক', 'অংশ', 'অগ্নিঝরা', 'অটোরিকশা', 'অটোরিকশায়', 'অদ্ভুদ', 'অধিকার', 'অনশন', 'অনির্বাণ', 'অনুপ্রেরণা', 'অন্ধত্বের ', 'অন্যায়', 'অপমান', 'অপেক্ষা', 'অবভ্যাস', 'অবহেলা', 'অভিজ্ঞতা', 'অভিনয়', 'অভিমান', 'অভিযান', 'অর্থ ', 'অর্থনীতির', 'অলিম্পিয়াড', 'অসৎ', 'অহংকার', 'অ্যাডেলের', 'অ্যান্ড্রয়েড', 'আ.লীগ', 'আক্রমণ', 'আক্রান্ত', 'আগামীকাল', 'আগুন', 'আড্ডা', 'আত্মসম্মান', 'আত্মহত্যা', 'আধুনিক', 'আবিস্কার', 'আমন্ত্রণ', 'আমাদের', 'আলো', 'আসিফ', 'ইউপি', 'ইঞ্জিনিয়ারিংয়ের ', 'ইতিহাস', 'ইতিহাসে', 'ইলেকট্রিক', 'ইয়াবা', 'উইন্ডোজে', 'উচ্ছেদ', 'উত্তম', 'উদ্দেশ্য', 'উপকার', 'উপভোগ', 'উপমন্ত্রীর ', 'উপসর্গ', 'উপস্থিতি', 'উৎসবে', 'একরকম', 'একাকিত্ব', 'এমপি', 'এলসিডি', 'এলাকা', 'ওয়ালটন', 'ককটেল ', 'কনফারেন্সের', 'কম্পিউটারে ', 'কম্পিউটারের', 

In [ ]:
print(documents[0])

(['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত'], 'নিহত')


In [ ]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [ ]:
for i in training:
  print(i[0],end=' ')
  print(i[1])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])


Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....


In [ ]:
train_x2 = train_x[0:1441]
train_y2 = train_y[0:1441]
#test_x2 = train_x[1210:1441]
#test_y2 = train_y[1210:1441]

In [ ]:
print(len(train_x2))

1441


In [ ]:
print(train_x2[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(train_y2[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(len(train_x2[0]))

1831


In [ ]:
print(len(train_y2[0]))

383


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 6)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=8,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(len(train_y2[0]), activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss= keras.losses.MeanAbsoluteError(),
        metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective = 'val_accuracy', 
                     max_epochs = 150,
                     factor = 3)

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(train_x2, train_y2, epochs = 80, validation_data = (train_x2, train_y2))


Trial 254 Complete [00h 01m 03s]
val_accuracy: 0.20194309949874878

Best val_accuracy So Far: 0.6759194731712341
Total elapsed time: 01h 49m 10s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 328
units_1: 168
learning_rate: 0.01
units_2: 328
units_3: 136
units_4: 136
units_5: 264
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 5635dc4c169e79661f31feabeb3fc21e
Score: 0.6759194731712341
Trial summary
Hyperparameters:
num_layers: 2
units_0: 264
units_1: 264
learning_rate: 0.001
units_2: 360
units_3: 456
units_4: 328
units_5: 104
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 3a3b7f39e7d5cf5bfb8feb359f548f5c
Score: 0.4462178945541382
Trial summary
Hyperparameters:
num_layers: 2
units_0: 328
units_1: 168
learning_rate: 0.01
units_2: 328
units_3: 136
units_4: 136
units_5: 264
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 3
tuner/round: 2
tuner/trial_id: 47accc45c646c0ccd42d49edd692df7f
Score: 0.43095073103904724
Trial 

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(train_x2, train_y2, epochs = 5000, validation_data = (train_x2, train_y2))

Streaming output truncated to the last 5000 lines.
Epoch 2501/5000
46/46 [==============================] - 0s 7ms/step - loss: 0.0019 - accuracy: 0.6415 - val_loss: 0.0019 - val_accuracy: 0.6440
Epoch 2502/5000
46/46 [==============================] - 0s 6ms/step - loss: 0.0019 - accuracy: 0.6438 - val_loss: 0.0019 - val_accuracy: 0.6440
Epoch 2503/5000
46/46 [==============================] - 0s 6ms/step - loss: 0.0019 - accuracy: 0.6292 - val_loss: 0.0019 - val_accuracy: 0.6440
Epoch 2504/5000
46/46 [==============================] - 0s 7ms/step - loss: 0.0018 - accuracy: 0.6586 - val_loss: 0.0019 - val_accuracy: 0.6440
Epoch 2505/5000
46/46 [==============================] - 0s 6ms/step - loss: 0.0018 - accuracy: 0.6562 - val_loss: 0.0019 - val_accuracy: 0.6440
Epoch 2506/5000
46/46 [==============================] - 0s 7ms/step - loss: 0.0019 - accuracy: 0.6379 - val_loss: 0.0019 - val_accuracy: 0.6440
Epoch 2507/5000
46/46 [==============================] - 0s 6ms/step - loss: 0.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 328)               600896    
_________________________________________________________________
dense_1 (Dense)              (None, 168)               55272     
_________________________________________________________________
dense_2 (Dense)              (None, 383)               64727     
Total params: 720,895
Trainable params: 720,895
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.evaluate(train_x2,train_y2)

46/46 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 0.6384


[0.0018880130955949426, 0.6384454965591431]

In [ ]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [ ]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('/content/gdrive/My Drive/Final defence/intents3.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
#model3.load('./model3.tflearn')

Loading Pickle.....
Loading the Model......


In [ ]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- মার্কিন যুক্তরাষ্ট্রের সরকার রাজনীতি ও নির্বাচনী ব্যবস্থা


ValueError: ignored